In [1]:
# Recursive Least Mean Squares

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from ipywidgets import interact, fixed
%matplotlib inline

import time

print("Packages imported successfully.")

def print_header(str):
    print("======================================================================")
    print(str)
    print("======================================================================")

Packages imported successfully.


In [2]:
print_header("Step 0: Generate random data to test on.")

num_samples  = 10000
num_features = 30
# Generate 2-D matrix: (num_samples) columns of (num_features) rows
init_input_data = np.matrix(
    np.random.rand(
        num_features,
        num_samples
    )
)
# Generate 1-D vector matrix: (num_features) weights -- 1 weight per features
init_weights = np.matrix(
    np.random.rand(
        num_features, 1
    )
)
# Genereate 1-D matrix: (num_samples) desired outputs -- 1 desired output per sampl
#   We are generating a "noisy linear" data set from the W.T * X basic model so that it's already linear...
#   plus an error/noise term. So our random data is based off of a linear relationship between the model
#   and the input.
# Not necessarily used below...
init_desired = np.matrix(
   (init_weights.T * init_input_data) + np.random.rand(1, num_samples)
)


print("shape(X) =", init_input_data.shape)
print("shape(W) =", init_weights.shape)
print("shape(D) =", init_desired.shape)

Step 0: Generate random data to test on.
shape(X) = (30, 10000)
shape(W) = (30, 1)
shape(D) = (1, 10000)


In [3]:
# Ordinary / One-Step / Iterative Least Mean Squares Linear Regression
t0 = time.time()

final_weights = 0;
X = init_input_data
D = init_desired
for i in range(1, num_samples):
    final_weights = ((X[:,0:i]*(X[:,0:i]).T).I*(X[:, 0:i]*(D[0,0:i]).T))

t1 = time.time()

In [4]:
print_header("Step 1: Create the n=1 iteration of the model first.")

x_1 = init_input_data[:, 0]
d_1 = init_desired[:, 0]
p_1 = (x_1 * x_1.T).I
w_1 = p_1 * (x_1 * d_1.T)

print("x(n=1) =", x_1)
print("d(n=1) =", d_1)
print("p(n=1) =", p_1)
print("w(n=1) =", w_1)


# w_weights
# x_next
# d_desired
# d_desired_next
# p_covar
# y_predicted
# g_gain_next
# w_weights_next
# alpha_innovation


Step 1: Create the n=1 iteration of the model first.
x(n=1) = [[ 0.26674084]
 [ 0.14416291]
 [ 0.91904902]
 [ 0.22442374]
 [ 0.31422142]
 [ 0.13688357]
 [ 0.54350374]
 [ 0.33837775]
 [ 0.21257628]
 [ 0.53980553]
 [ 0.44944249]
 [ 0.77846603]
 [ 0.07815337]
 [ 0.20435283]
 [ 0.18708085]
 [ 0.73439788]
 [ 0.02839123]
 [ 0.9397057 ]
 [ 0.82110767]
 [ 0.8529785 ]
 [ 0.76650222]
 [ 0.27677744]
 [ 0.76574291]
 [ 0.19006768]
 [ 0.61710351]
 [ 0.04365514]
 [ 0.64426828]
 [ 0.23315567]
 [ 0.1964533 ]
 [ 0.13588775]]
d(n=1) = [[ 7.3004657]]
p(n=1) = [[  4.18672649e+16  -1.02488914e+17   2.66711819e+15   1.10016879e+16
   -2.05146986e+16   6.66429917e+16  -2.21066617e+16  -1.42528960e+15
    9.67601673e+16   1.06493221e+16   3.38201736e+16   5.56605726e+15
   -1.32186695e+16   3.91038806e+16  -8.04352895e+16  -1.25385974e+16
   -2.74260735e+17   1.44939557e+16   7.13569980e+15  -1.29237731e+16
   -2.36426211e+15   1.43661197e+16  -6.54888678e+15  -4.84159904e+16
   -3.23693450e+15  -2.69264119e+1

In [5]:
print_header("Step 2: Start training on the data, one sample at a time.")

print("Starting from n = 0", "to num_samples =", num_samples)

t2 = time.time()

# Assuming we have p_n and w_n such that (n) = the current iteration
# we want to generate the (next) iteration for the model.
x_n = x_1
d_n = d_1
p_n = p_1
w_n = w_1

for i in range(1,num_samples):
    # Get the next input/output sample pair (a vector).
    x_next = init_input_data[:, i]
    d_next = init_desired[:, i]
    
    # Calculate alpha_next, the innovation matrix for the (n+1) case.
    alpha_next = (d_next) - (w_n.T * x_next).item(0)
    
    # Calculate gain_next, the gain vector for the matrix.
    gain_next = (p_n * x_next) * (1 + (x_next.T * p_n * x_next)).I
    
    # Calculate p_next, the estimated covariance matrix.
    p_next = (p_n) - (gain_next * x_next.T * p_n)
    
    # Calculate w_next, the weights matrix for the next iteration
    w_next = (w_n) + (gain_next * alpha_next)
    
    # Finally, make sure that the next iteration becomes the current iteration
    # before we start the next iteraton (as next -> current)
    p_n = p_next
    w_n = w_next
    
t3 = time.time()

Step 2: Start training on the data, one sample at a time.
Starting from n = 0 to num_samples = 10000


In [6]:
print_header("Step 3: Print out our final model!")

print("Final p (estimated covariance matrix: ", p_n)

Step 3: Print out our final model!
Final p (estimated covariance matrix:  [[  1.17001435e-03  -3.31042587e-05  -6.48933401e-05  -3.27639902e-05
   -4.50912718e-05   5.22986718e-07  -2.74184315e-05  -4.54351569e-05
   -6.03719298e-05  -2.95472603e-05  -4.22204567e-05  -4.55984396e-05
   -2.97985346e-05  -4.69162662e-05  -3.30233663e-05  -4.44131231e-05
   -4.74890998e-05  -3.98502664e-05  -3.71554218e-05  -4.82050727e-05
   -4.03612963e-05  -4.56074706e-05  -5.46318649e-05  -5.17537591e-05
   -1.49874372e-05  -4.32527513e-05  -2.22970829e-05  -5.20422705e-05
   -4.05940771e-05  -4.17047842e-05]
 [ -3.31791886e-05   1.17018545e-03  -2.90735694e-05  -5.63496116e-05
   -2.99736176e-05  -3.41439753e-05  -4.08997213e-05  -4.42243725e-05
   -4.30432268e-05  -5.61348635e-05  -4.17883373e-05  -5.02478570e-05
   -2.32161147e-05  -5.22365210e-05  -6.19702724e-05  -3.80604195e-05
   -1.41089619e-05  -3.37743068e-05  -2.69899476e-05  -1.43025763e-05
   -4.58749243e-05  -2.77918182e-05  -5.03886256e

In [7]:
print("Final w (weights matrix):", w_n)

Final w (weights matrix): [[ 0.69212236]
 [ 0.65755649]
 [ 0.50696383]
 [ 0.91942712]
 [ 0.67971561]
 [ 0.28094716]
 [ 0.82255489]
 [ 0.04335995]
 [ 0.05874822]
 [ 0.47533883]
 [ 0.04813013]
 [ 0.62944874]
 [ 0.13363742]
 [ 0.40008084]
 [ 0.58022322]
 [ 0.53672665]
 [ 0.71576114]
 [ 0.33083894]
 [ 0.96622811]
 [ 0.76990493]
 [ 0.83919764]
 [ 0.89054728]
 [ 0.16901083]
 [ 0.16512983]
 [ 0.57143896]
 [ 0.65787363]
 [ 0.06995773]
 [ 0.79475462]
 [ 0.90245201]
 [ 0.7115324 ]]


In [8]:
print("Final iterative w (weights matrix)", final_weights)

Final iterative w (weights matrix) [[ 0.69128242]
 [ 0.65594187]
 [ 0.50702638]
 [ 0.92006304]
 [ 0.68088414]
 [ 0.28111949]
 [ 0.82227947]
 [ 0.04246841]
 [ 0.06071914]
 [ 0.47551589]
 [ 0.04785273]
 [ 0.63167312]
 [ 0.13466079]
 [ 0.40083156]
 [ 0.57909939]
 [ 0.53647564]
 [ 0.71351249]
 [ 0.33020999]
 [ 0.9669621 ]
 [ 0.77034768]
 [ 0.83956189]
 [ 0.88965445]
 [ 0.17023278]
 [ 0.16426048]
 [ 0.57073672]
 [ 0.65924933]
 [ 0.0694694 ]
 [ 0.79446904]
 [ 0.90186294]
 [ 0.71130939]]


In [9]:
print_header("FINAL TIME COMPARISON:")
print("Iterative (seconds): ", t1-t0)
print("Recursive (seconds): ", t3-t2)

FINAL TIME COMPARISON:
Iterative (seconds):  22.828705310821533
Recursive (seconds):  1.7562744617462158
